# Perdiction Pipeline | Ensemble Pipeline

In this notebook, we will ensemble our models to make a prediction pipeline.
The keys steps would be as following :

  1. Text Preprocessing for inference.
  2. Load classifiers iteratively in a list.
  3. Load test-data & pre-process.
  4. Set classifier threshold and run it through Ensemble



In [2]:
# First let's check what has Google given us ! Thank you Google for the GPU

!nvidia-smi

Tue Dec 31 14:14:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
# Let's mount our G-Drive. Hey !! Because for GPU you now give your data to Google 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Install necessary packages and restart the environment

!pip install tiny-tokenizer
!pip install  flair

  Created wheel for tiny-tokenizer: filename=tiny_tokenizer-3.1.0-cp36-none-any.whl size=10550 sha256=58a4751f88c984e23a7342dd0be9729e53dabcc86c46e98791474c4081c2ab20
  Stored in directory: /root/.cache/pip/wheels/d1/c8/36/334497a689fab90128232e86b5829b800dd271a3d5d5959c53
Successfully built tiny-tokenizer
     |████████████████████████████████| 194kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 71.1MB/s 
     |████████████████████████████████| 798kB 58.8MB/s 
     |████████████████████████████████| 778kB 52.8MB/s 
     |████████████████████████████████| 450kB 57.4MB/s 
     |████████████████████████████████| 3.8MB 45.0MB/s 
     |████████████████████████████████| 21.5MB 152kB/s 
     |████████████████████████████████| 70.7MB 36kB/s 
     |████████████████████████████████| 1.0MB 52.4MB/s 
     |████████████████████████████████| 81kB 13.8MB/s 
     |████████████████████████████████| 348kB 73.9MB/s 
     |████████████████████████████████| 860kB 61.2MB/s 
  Created wheel for lan

In [1]:
# Let's import our packages !

import pandas as pd
from tqdm import tqdm
import html
import re
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
# import flair
import pickle
from torch.optim.adam import Adam

# The sentence objects holds a sentence that we may want to embed or tag
from flair.data import Sentence
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.samplers import ImbalancedClassificationDatasetSampler

# 1. Text Pre-processing

### Note -
We need to be careful to apply the same set of text transformations during inference as we did during training. Any changes will directly affect the model to produce junk / more junky results

In [0]:
clean = re.compile('<.*?>')

def preprocess_text(text) :
  try :
    # soup = BeautifulSoup(text, "html.parser")
    # text = soup.get_text()
    text=  re.sub(clean, '', text)
    text = html.unescape(text)
  except :
    print("Error in HTML Processing ...")
    print(text)
    text = text
  try :
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
  except :
    print("Error in removing extra lines ...")
    print(text)
    text = text

  try :
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
  except :
    print("Error in extra whitespace removal ...")
    print(text)
    text = text

  return text

# 2. Load Classifiers

We iterate through the saved classifiers and load them. We append them in a list can be easily iterated during evaluation/prediction.

In [7]:
classifiers  = []

for grp_id in tqdm(range(1,15)) :
  path  = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised/group/' + str(grp_id) + '/best-model.pt'
  classifier = TextClassifier.load(path)
  classifiers.append(classifier)

  0%|          | 0/14 [00:00<?, ?it/s]

2019-12-31 15:28:13,097 loading file /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/1/best-model.pt


  7%|▋         | 1/14 [00:22<04:51, 22.40s/it]

2019-12-31 15:28:35,497 loading file /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/2/best-model.pt


 14%|█▍        | 2/14 [00:47<04:37, 23.13s/it]

...

 86%|████████▌ | 12/14 [05:21<00:58, 29.24s/it]

2019-12-31 15:33:34,520 loading file /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/13/best-model.pt


 93%|█████████▎| 13/14 [05:52<00:29, 29.69s/it]

2019-12-31 15:34:05,240 loading file /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/14/best-model.pt


100%|██████████| 14/14 [06:27<00:00, 31.37s/it]


# 3. Load Test Data & Preprocess

We load the test-data from the previously created split and re-transform it into a Dataframe with corresponding labels.

Finally we save the DataFrame with a placeholder for the predicted labels.

In [0]:
test_data = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/70_30_split/test.txt'

with open(test_data,'r',encoding='utf-8') as f :
  data = f.readlines()

# prefix from the Training Data Format
prefix = '__label__'

In [0]:
test_text = []
test_label = []
for doc in tqdm(data) :
  splits =  doc.split()
  labels = []
  idx = 0 
  for word in splits :
    if prefix in word :
      labels.append(word[9:].strip())
      idx += len(word)
    else :
      text = doc[idx:].strip()
      break
  test_text.append(text)
  test_label.append(labels)
  # break

test_df = pd.DataFrame(list(zip(test_text,test_label)),columns = ['text','original_labels'])
test_df['predicted_labels'] = None


100%|██████████| 157799/157799 [00:02<00:00, 71856.09it/s]


In [0]:
test_df.to_pickle('/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/70_30_split/test.pkl')
test_df.head()

# 4. Run the Ensemble Pipeline

Once we load the test-data, we can iteratively run it via all the classifiers and store the predictions for evaluating the performance.

### Coding Exercise # X
  1. Multi-thread/process the prediction pipeline.
  2. Find a redundant line of code in the cell below
  3. Optimize it

In [0]:
# Read the cleaned Test Data for running with different classifiers & threshold

with open('/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/70_30_split/test.pkl','rb') as f :
  test_df = pickle.load(f)

In [9]:
## Setting a global threshold for all classifiers
threshold = 0.9

## Iterating through each Test Example
for idx in tqdm(test_df.index) :
  text = preprocess_text(test_df.loc[idx,'text'])
  
  # create example sentence
  sentence = Sentence(text)
  labels = []

  ## Iterate through each classifier or prediction
  for classifier in classifiers:
    classifier.multi_label_threshold = threshold
    classifier.predict(sentence)

    for label in sentence.labels :
      ## Append labels from all classifiers
      labels.append(label.value)

  test_df.at[idx,'predicted_labels'] = labels


 85%|████████▌ | 134329/157799 [4:09:11<53:03,  7.37it/s]

Buffered data was truncated after reaching the output size limit.

In [10]:
# Save the predicteions
test_df.to_pickle('/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/70_30_split/normalised_test_predcted_th_0'+ str(int(threshold*10)) +'.pkl')
test_df.head()

,text,original_labels,predicted_labels
0,More Efficient way to code if then else statem...,[c++],[testing]
1,"p\t""Make appear at current scroll position. Ok...","[javascript, php]","[go, testing, tdd]"
2,"""Symbol(s) not found for architecture i386 - M...",[xcode],[]
3,Google PlusOneButton not compatible with Insta...,[android],"[android, testing]"
4,"on\t""How to install MySql-python-1.2.3 on Mac ...","[osx, mysql, python]",[testing]
